In [17]:
import matplotlib.pyplot as plt
import numpy as np

from gridmap import GridMap

In [48]:
#CONFIGURATIONS

directions = {
    'North': np.pi/2,
    'East': 2*np.pi,
    'South': (3*np.pi)/2,
    'West': np.pi,
    
    'Right': 2*np.pi,
    'Left' : np.pi
}

# Expected orientation of robot when facing obstacle
robotPositions = {
    'N': directions['South'],
    'E': directions['West'],
    'S': directions['North'],
    'W': directions['East'],
    
    'R': directions['Right'],
    'L': directions['Left'],
}

# Positions RC is expected to be in to accurately capture the image of the obstacles
# Obstacle Orientation to RC Position
# North:    Y-Axis + 2, Directions to be South
# East:     X-Axis + 2, Directions to be West
# South:    Y-Axis - 2, Directions to be North
# West:     X-Axis - 2, Directions to be East
def expectedPos(obstacle):
    obsOrient = obstacle[2]
    ax = obstacle[0]
    ay = obstacle[1]

    if obsOrient == 'N': ay += 2
    elif obsOrient == 'E': ax += 2
    elif obsOrient == 'S': ay -= 2
    else: ax -= 2
    
    orient = robotPositions[obsOrient]
    finalPos = (ax, ay, orient)
    return finalPos

In [58]:
# Tester values

obstacleList = [(6, 9, 'R'),
                (13, 9, 'L')]

initPosition = (0, 9, directions['East'])

# First element represents the inital position the RC will be in
positions = [initPosition]

for obs in obstacleList:
    positions.append(expectedPos(obs))

# Possible positions RC will complete its pathing in
finalPositions = [(0, 8, directions['East']),
                  (0, 8, directions['West'])]

In [ ]:
""" add parking spot (walls) """

# def parkingWall(obstacle):
#     obsOrient = obstacle[2]
#     ax = obstacle[0]
#     ay = obstacle[1]

#     if obsOrient == 'N': ay += 2
#     elif obsOrient == 'E': ax += 2
#     elif obsOrient == 'S': ay -= 2
#     else: ax -= 2
    
#     orient = robotPositions[obsOrient]
#     finalPos = (ax, ay, orient)
#     return finalPos

# parkingSpot = [(0, 8, '-'), (0,10, '-')]

# for park in parkingSpot:
#     positions.append(parkingWall(obs))

In [56]:
map = GridMap([20, 20])
for ob in obstacleList:
    map.setObstacles(ob)

In [62]:
maze = map.printGrid()


['.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.']
['.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.']
['.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.']
['.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.']
['.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.']
['.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.']
['.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.']
['.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.']
['.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.']
['.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', 

In [69]:
class Node():

    def __init__(self, parent=None, position=None):
        self.parent = parent
        self.position = position

        self.g = 0
        self.h = 0
        self.f = 0

    def __eq__(self, other):
        return self.position == other.position
    
    def astar(maze, start, end):
        """Returns a list of tuples as a path from the given start to the given end in the given maze"""

        # Create start and end node
        start_node = (None, start)
        start_node.g = start_node.h = start_node.f = 0
        end_node = (None, end)
        end_node.g = end_node.h = end_node.f = 0

        # Initialize both open and closed list
        open_list = []
        closed_list = []

        # Add the start node
        open_list.append(start_node)

        # Loop until you find the end
        while len(open_list) > 0:

            # Get the current node
            current_node = open_list[0]
            current_index = 0
            for index, item in enumerate(open_list):
                if item.f < current_node.f:
                    current_node = item
                    current_index = index

            # Pop current off open list, add to closed list
            open_list.pop(current_index)
            closed_list.append(current_node)

            # Found the goal
            if current_node == end_node:
                path = []
                current = current_node
                while current is not None:
                    path.append(current.position)
                    current = current.parent
                return path[::-1]  # Return reversed path

            # Generate children
            children = []
            for new_position in [(0, -1), (0, 1), (-1, 0), (1, 0), (-1, -1), (-1, 1), (1, -1), (1, 1)]:  # Adjacent squares
                """ 
                In the tuple (0, -1), the first value (0) represents the change in the row index, 
                and the second value (-1) represents the change in the column index. 
                So, (0, -1) corresponds to moving one step to the left.
                
                (0, -1) represents moving one step left,
                (0, 1) represents moving one step right,
                (-1, 0) represents moving one step up,
                (1, 0) represents moving one step down,
                (-1, -1) represents moving one step diagonally up-left,
                (-1, 1) represents moving one step diagonally up-right,
                (1, -1) represents moving one step diagonally down-left,
                (1, 1) represents moving one step diagonally down-right.
                """

                # Get node position
                node_position = (current_node.position[0] + new_position[0], current_node.position[1] + new_position[1])

                # Make sure within range
                if node_position[0] > (len(maze) - 1) or node_position[0] < 0 or node_position[1] > (
                        len(maze[len(maze) - 1]) - 1) or node_position[1] < 0:
                    continue

                # Make sure walkable terrain
                if maze[node_position[0]][node_position[1]] != 0:
                    continue

                # Create new node
                new_node = Node(current_node, node_position)

                # Append
                children.append(new_node)

            # Loop through children
            for child in children:

                # Child is on the closed list
                for closed_child in closed_list:
                    if child == closed_child:
                        continue

                # Create the f, g, and h values
                child.g = current_node.g + 1
                child.h = ((child.position[0] - end_node.position[0]) ** 2) + (
                            (child.position[1] - end_node.position[1]) ** 2)
                child.f = child.g + child.h

                # Child is already in the open list
                for open_node in open_list:
                    if child == open_node and child.g > open_node.g:
                        continue

                # Add the child to the open list
                open_list.append(child)

In [70]:
def visualize_path(maze, path):
    """Visualize the path on the grid map"""
    for row in range(len(maze)):
        for col in range(len(maze[row])):
            if (row, col) == path[0]:
                print("S", end=" ")  # Start
            elif (row, col) == path[-1]:
                print("E", end=" ")  # End
            # elif (row, col) in path:
            #     print("*", end=" ")  # Path
            # elif maze[row][col] == 1:
            #     print("#", end=" ")  # Obstacle
            # else:
            #     print(".", end=" ")  # Open space
        print()

In [71]:
start = (0, 9)
end = (0, 8)

path = astar(maze, start, end)
visualize_path(maze, path)

AttributeError: 'tuple' object has no attribute 'g'